In [3]:
from music21 import *
import pandas as pd
import numpy as np

### Features
- chord_names (nominal)
        - Label encoding (C major -> 0, A major -> 13)
        - One-hot encoding?
        - Feature hashing
- roman_numerals

In [ ]:
myChord = chord.Chord(["E-5","C4","G4"])
myChord.show()

In [ ]:
myChord

In [ ]:
pitches = myChord.pitches
pitches

In [ ]:
melody = pitches[-1]
melody

In [ ]:
myChord.sortAscending(inPlace = True)

In [ ]:
bachCorpus = corpus.search('bach', 'composer')
bachCorpus[-23].parse().show()

In [ ]:
#Load corpus of Bach chorales
bachCorpus = corpus.search('bach', 'composer')


for chorale in bachCorpus:
    chorale = chorale.parse()
    print('Processing chorale: {}'.format(chorale.metadata.title))
    sopranoPart = chorale.getElementsByClass('Part')[0]
    
    for keychange in sopranoPart.recurse().getElementsByClass(key.Key):
        #Get key information and offset location
        print(keychange, keychange.offset)


In [ ]:
#Iterate through soprano part and print all key changes
print('Key Changes:')
for keychange in soprano.recurse().getElementsByClass(key.Key):
    #Get key information and offset location
    print(keychange, keychange.offset)

In [ ]:
bachFeatures = pd.DataFrame(np.zeros((len(bach),2)), columns = ['roman_numerals', 'mel_notes'], dtype='object')
bachFeatures

In [ ]:
bachFeatures.dtypes

In [ ]:
k = key.Key('f#')  # 3-sharps minor
rn = roman.RomanNumeral('V', k)
rn.key

In [ ]:
rn.pitchedCommonName

### Feature Extraction

In [85]:
# extract the notes from each piece in the corpus and place in in the x_train variable

bachCorpus = corpus.getComposer('bach')
x_train = []
# gets the index to change to 1 in the feature vector when a given value of pitch class is encountered
pc_dict = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10, 'B': 11}
# gets the index to change to 1 in the feature vector when a given value of octave is encountered
oct_dict = {0: 12, 1: 13, 2: 14, 3: 15, 4: 16, 5: 17, 6: 18, 7: 19}

durations = []

# TODO: extract rests as well
# TODO: bin durations

for piece in bachCorpus:
    sample = []
    pieceScore = corpus.parse(piece)  # stream.Score object
    melodyPart = pieceScore.getElementsByClass('Part')[0]  # melody parts are always the first part in the score
    melodyNotes = melodyPart.flat.getElementsByClass('Note')
    
    # DON'T CHANGE n to Note or note (namespace conflict)
    # builds the feature vector by One-Hot encoding pitch class/octave with the duration at the end
    for n in melodyNotes.recurse():  # iterates through all notes in the piece
        pitchClass = n.pitch.pitchClassString
        pitchOctave = n.pitch.octave
        noteDuration = n.duration.quarterLength
        if noteDuration not in durations:
            durations.append(noteDuration)
        new_row = np.zeros(21) # initialize this note's feature vector
        new_row[pc_dict[pitchClass]] = 1
        new_row[oct_dict[pitchOctave]] = 1
        new_row[-1] = noteDuration
        sample.append(new_row) # appends the next time step to the sample
        
    x_train.append(sample)

In [90]:
durations

[1.0, 0.5, 0.25, 2.0, 1.5, 4.0, 3.0, 0.75, 8.0, 6.0, 0.125, 0.0]

In [87]:
# set up labels for each time step of each piece (which is just the next note)

SyntaxError: invalid syntax (<ipython-input-87-c903ed16cfc7>, line 4)